# Dự đoán điểm từ dữ liệu nộp bài wecode
## Thành viên nhóm
- Phan Thanh Đăng - 22520193
- Phùng Văn Đạt - 22520234
- Lê Văn Giáp - 22520363

### Importing libraries

In [44]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error, r2_score
import warnings
warnings.filterwarnings('ignore')

import csv, glob

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Load data

In [22]:
df = pd.read_csv('drive/MyDrive/CS114/Competition_2/data_tbtl/annonimized.csv', header=None)
df = df.drop(0)
df.columns = ['assign_id', 'prob_id', 'stud_id', 'is_final', 'status', 'score', 'coefficient', 'subj_id', 'created_at', 'updated_at', 'judgements']
df.head(10)

,assign_id,prob_id,stud_id,is_final,status,score,coefficient,subj_id,created_at,updated_at,judgements
1,90ce27571176d87961b565d5ef4b3de33ede04ac,789454427dd4097a14749e3dde63346b7a8d3811,ed9eaeb6a707f50154024b24d7efcb874a9795dd,0,SCORE,0,100,it0012,10-09 08:02:04,10-09 08:06:58,"{""times"":[0,0,0,0,0,0,0,0,0,0],""mems"":[0,0,0,0..."
2,90ce27571176d87961b565d5ef4b3de33ede04ac,789454427dd4097a14749e3dde63346b7a8d3811,ed9eaeb6a707f50154024b24d7efcb874a9795dd,0,SCORE,0,100,it0012,10-09 08:04:41,10-09 08:04:51,"{""times"":[0,0,0,0,0,0,0,0,0,0],""mems"":[0,0,0,0..."
3,90ce27571176d87961b565d5ef4b3de33ede04ac,789454427dd4097a14749e3dde63346b7a8d3811,ed9eaeb6a707f50154024b24d7efcb874a9795dd,1,SCORE,10000,100,it0012,10-09 08:06:49,10-09 08:06:58,"{""times"":[0,0,0,0,0,0,0,0,0,0],""mems"":[0,0,0,0..."
4,90ce27571176d87961b565d5ef4b3de33ede04ac,bf96fbdc5f499538c3e2bfbec5779c8a14b0a9ff,ed9eaeb6a707f50154024b24d7efcb874a9795dd,1,SCORE,10000,100,it0012,10-09 08:47:52,10-09 08:48:01,"{""times"":[0,0,0,0,0,0,0,0,0,0],""mems"":[0,0,0,0..."
5,90ce27571176d87961b565d5ef4b3de33ede04ac,7a6e5ca470ff47c3b5048f240c4738de71010c78,ed9eaeb6a707f50154024b24d7efcb874a9795dd,1,SCORE,10000,100,it0012,10-09 09:19:35,10-09 09:19:45,"{""times"":[0,0,0,0,0,0,0,0,0,0],""mems"":[0,0,0,0..."
6,90ce27571176d87961b565d5ef4b3de33ede04ac,d6a5bdd7f7f32364f4743854ef3b5369c0706677,ed9eaeb6a707f50154024b24d7efcb874a9795dd,0,Compilation Error,0,100,it0012,10-09 09:29:03,10-09 09:30:08,"{""times"":[],""mems"":[],""verdicts"":{""\n\nIn func..."
7,90ce27571176d87961b565d5ef4b3de33ede04ac,d6a5bdd7f7f32364f4743854ef3b5369c0706677,ed9eaeb6a707f50154024b24d7efcb874a9795dd,1,SCORE,10000,100,it0012,10-09 09:29:58,10-09 09:30:08,"{""times"":[0,0,0,0,0,0,0,0,0,0],""mems"":[0,0,0,0..."
8,90ce27571176d87961b565d5ef4b3de33ede04ac,8b15ab090e1e8ec3284cf78746d94ffa00bd201b,ed9eaeb6a707f50154024b24d7efcb874a9795dd,0,Compilation Error,0,100,it0012,10-09 09:32:17,10-09 09:42:54,"{""times"":[],""mems"":[],""verdicts"":{""\n\nforbidd..."
9,90ce27571176d87961b565d5ef4b3de33ede04ac,8b15ab090e1e8ec3284cf78746d94ffa00bd201b,ed9eaeb6a707f50154024b24d7efcb874a9795dd,0,Compilation Error,0,100,it0012,10-09 09:33:36,10-09 09:33:36,"{""times"":[],""mems"":[],""verdicts"":{""\n\nforbidd..."
10,90ce27571176d87961b565d5ef4b3de33ede04ac,8b15ab090e1e8ec3284cf78746d94ffa00bd201b,ed9eaeb6a707f50154024b24d7efcb874a9795dd,0,Compilation Error,0,100,it0012,10-09 09:34:05,10-09 09:34:05,"{""times"":[],""mems"":[],""verdicts"":{""\n\nforbidd..."


In [23]:
df.isna().sum()

assign_id      0
prob_id        0
stud_id        0
is_final       0
status         0
score          0
coefficient    0
subj_id        0
created_at     0
updated_at     0
judgements     0
dtype: int64

## Feature Engineering

In [25]:
# Lấy điểm cao nhất cho mỗi sinh viên trên mỗi problem
df_max_scores = df.groupby(['stud_id', 'assign_id', 'prob_id'])['score'].max().reset_index()

# Tính toán tổng số assignment mà mỗi sinh viên đã tham gia
total_assignments = df_max_scores.groupby('stud_id')['assign_id'].nunique().reset_index()
total_assignments.columns = ['student_id', 'total_assignments']

# Tính toán số problem mà mỗi sinh viên đã làm trong mỗi assignment
problems_per_assignment = df_max_scores.groupby(['stud_id', 'assign_id'])['prob_id'].nunique().reset_index()
average_problems = problems_per_assignment.groupby('stud_id')['prob_id'].mean().reset_index()
average_problems.columns = ['student_id', 'aver_problems']

df_max_scores['score'] = pd.to_numeric(df_max_scores['score'], errors='coerce')

# Tính toán điểm trung bình cho mỗi assignment và điểm trung bình toàn bộ các assignment
assign_avg_scores = df_max_scores.groupby(['stud_id', 'assign_id'])['score'].mean().reset_index()
overall_avg_score = assign_avg_scores.groupby('stud_id')['score'].mean().reset_index()
overall_avg_score.columns = ['student_id', 'aver_score']

# Hợp nhất các bảng thành một dataframe duy nhất
data = total_assignments.merge(average_problems, on='student_id')
data = data.merge(overall_avg_score, on='student_id')

data['aver_score'] /= 100
data['aver_score']= data['aver_score'].round(2)
data['aver_problems'] = data['aver_problems'].round(2)

# Hiển thị dataframe kết quả
data.head()

,student_id,total_assignments,aver_problems,aver_score
0,00b6dd4fc7eb817e03708c532016ef30ce564a61,7,6.71,100.00
1,00bef8afee8f3c595d535c9c03c490cac1a4f021,9,8.67,93.78
2,01122b3ef7e59b84189e65985305f575d6bdf83c,7,9.71,90.21
3,0134f9f410c65ad0e8c2254a7e9288670e02a183,4,11.75,100.00
4,013de369c439ab0ead8aa7da64423aa395a8be39,8,8.25,78.41


In [26]:
data.shape

(1489, 4)

## Prepare Training and Testing Data

In [29]:
df_qt = pd.read_csv('drive/MyDrive/CS114/Competition_2/public_it001/qt-public.csv')
df_qt.columns = ['student_id', 'QT']
df_qt.head()

,student_id,QT
0,00b6dd4fc7eb817e03708c532016ef30ce564a61,7.5
1,00bef8afee8f3c595d535c9c03c490cac1a4f021,7
2,01122b3ef7e59b84189e65985305f575d6bdf83c,9
3,013de369c439ab0ead8aa7da64423aa395a8be39,10
4,014c59c6433fd764a0b08de6ffeb757eaf60aa73,9


In [56]:
df_train = df_qt.merge(data, on='student_id', how='left')
df_train['QT'] = pd.to_numeric(df_train['QT'], errors='coerce')
df_train['total_assignments'] = pd.to_numeric(df_train['total_assignments'], errors='coerce')
df_train['aver_problems'] = pd.to_numeric(df_train['aver_problems'], errors='coerce')
df_train['aver_score'] = pd.to_numeric(df_train['aver_score'], errors='coerce')
df_train.head()

,student_id,QT,total_assignments,aver_problems,aver_score
0,00b6dd4fc7eb817e03708c532016ef30ce564a61,7.5,7,6.71,100.00
1,00bef8afee8f3c595d535c9c03c490cac1a4f021,7.0,9,8.67,93.78
2,01122b3ef7e59b84189e65985305f575d6bdf83c,9.0,7,9.71,90.21
3,013de369c439ab0ead8aa7da64423aa395a8be39,10.0,8,8.25,78.41
4,014c59c6433fd764a0b08de6ffeb757eaf60aa73,9.0,9,10.00,80.89


In [57]:
df_train.isna().sum()

student_id           0
QT                   7
total_assignments    0
aver_problems        0
aver_score           0
dtype: int64

In [58]:
df_train = df_train.dropna()

In [59]:
df_test = data[~data['student_id'].isin(df_train['student_id'])].reset_index(drop=True)
df_test['total_assignments'] = pd.to_numeric(df_test['total_assignments'], errors='coerce')
df_test['aver_problems'] = pd.to_numeric(df_test['aver_problems'], errors='coerce')
df_test['aver_score'] = pd.to_numeric(df_test['aver_score'], errors='coerce')
df_test.head()

,student_id,total_assignments,aver_problems,aver_score
0,0134f9f410c65ad0e8c2254a7e9288670e02a183,4,11.75,100.00
1,035f97702f2c01d26ab1fae8f39ea2f98a0caa3c,4,12.50,100.00
2,06024da8147b5cf6105beb4c403f67cf1ff0aea1,1,5.00,88.46
3,0aaebc88f6106684d6993c156104c1ef36cf94e0,4,12.50,100.00
4,0bf111a9caedf02804f6991792490e63bc21058a,6,13.67,92.13


In [60]:
df_test.isna().sum()

student_id           0
total_assignments    0
aver_problems        0
aver_score           0
dtype: int64

In [61]:
X_train, X_test, y_train, y_test = train_test_split(df_train[['total_assignments', 'aver_problems', 'aver_score']], df_train[['QT']], test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

impute = SimpleImputer(strategy='mean')
X_train_scaled = impute.fit_transform(X_train_scaled)
X_test_scaled = impute.transform(X_test_scaled)

## Model

In [62]:
models = {
  'Linear Regression': LinearRegression(),
  'Ridge Regression': Ridge(alpha=1.0),
  'Lasso Regression': Lasso(alpha=0.1),
  'Random Forest': RandomForestRegressor(n_estimators=100, random_state=42),
  'Gradient Boosting': GradientBoostingRegressor(n_estimators=100, random_state=42)
}

# Huấn luyện và đánh giá các mô hình
best_model = None
best_score = -np.inf

for name, model in models.items():
  model.fit(X_train_scaled, y_train)
  y_pred = model.predict(X_test_scaled)
  score = r2_score(y_test, y_pred)
  print(f'{name} R² score: {score:.4f}')
  if score > best_score:
    best_score = score
    best_model = model

Linear Regression R² score: -0.0462
Ridge Regression R² score: -0.0461
Lasso Regression R² score: -0.0253
Random Forest R² score: -0.1990
Gradient Boosting R² score: -0.1710


## Predict

In [63]:
path = "drive/MyDrive/CS114/Competition_2/result_qt.csv"

with open(path, 'w', newline='') as file:
  writer = csv.writer(file)
  for _, row in df_test.iterrows():
    y_pred = model.predict([[row['total_assignments'], row['aver_problems'], row['aver_score']]])
    writer.writerow([row['student_id'], y_pred[0].round(2)])

## Dự đoán điểm thực hành

### Create one more feature

In [64]:
# Điểm quá trình được sử dụng như là một feature để dự đoán điểm thực hành
df_th = pd.read_csv('drive/MyDrive/CS114/Competition_2/public_it001/th-public.csv')
df_th.columns = ['student_id', 'TH']

df_train = df_th.merge(data, on='student_id', how='left')
df_train['TH'] = pd.to_numeric(df_train['TH'], errors='coerce')
df_train['total_assignments'] = pd.to_numeric(df_train['total_assignments'], errors='coerce')
df_train['aver_problems'] = pd.to_numeric(df_train['aver_problems'], errors='coerce')
df_train['aver_score'] = pd.to_numeric(df_train['aver_score'], errors='coerce')
df_train.head()

df_train = df_train.dropna()

df_test = data[~data['student_id'].isin(df_train['student_id'])].reset_index(drop=True)
df_test['total_assignments'] = pd.to_numeric(df_test['total_assignments'], errors='coerce')
df_test['aver_problems'] = pd.to_numeric(df_test['aver_problems'], errors='coerce')
df_test['aver_score'] = pd.to_numeric(df_test['aver_score'], errors='coerce')

In [66]:
X_train, X_test, y_train, y_test = train_test_split(df_train[['total_assignments', 'aver_problems', 'aver_score']], df_train[['TH']], test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

impute = SimpleImputer(strategy='mean')
X_train_scaled = impute.fit_transform(X_train_scaled)
X_test_scaled = impute.transform(X_test_scaled)

models = {
  'Linear Regression': LinearRegression(),
  'Ridge Regression': Ridge(alpha=1.0),
  'Lasso Regression': Lasso(alpha=0.1),
  'Random Forest': RandomForestRegressor(n_estimators=100, random_state=42),
  'Gradient Boosting': GradientBoostingRegressor(n_estimators=100, random_state=42)
}

# Huấn luyện và đánh giá các mô hình
best_model = None
best_score = -np.inf

for name, model in models.items():
  model.fit(X_train_scaled, y_train)
  y_pred = model.predict(X_test_scaled)
  score = r2_score(y_test, y_pred)
  print(f'{name} R² score: {score:.4f}')
  if score > best_score:
    best_score = score
    best_model = model

Linear Regression R² score: 0.0753
Ridge Regression R² score: 0.0754
Lasso Regression R² score: 0.0747
Random Forest R² score: 0.1087
Gradient Boosting R² score: 0.1726


In [67]:
path = "drive/MyDrive/CS114/Competition_2/result_th.csv"

with open(path, 'w', newline='') as file:
  writer = csv.writer(file)
  for _, row in df_test.iterrows():
      y_pred = model.predict([[row['total_assignments'], row['aver_problems'], row['aver_score']]])
      writer.writerow([row['student_id'], min(y_pred[0].round(2), 10)])